In [3]:
import random
from sklearn.datasets import fetch_openml
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score 

In [4]:
x, y = fetch_openml('mnist_784', version=1, return_X_y=True) #importing data
x = (x/255).astype('float32')
y = tf.keras.utils.to_categorical(y)

x_train,x_test,y_train,y_test=train_test_split(x, y, test_size=0.15, random_state=38) #splitting it into training and testing data

In [20]:
size_il=784
size_ol=10
size_hl=[256, 64]

In [21]:
#initialising random bias and weights
random.seed(1)
W1=np.random.randn(size_hl[0], size_il) * np.sqrt(1/size_hl[0])
W2=np.random.randn(size_hl[1], size_hl[0]) * np.sqrt(1/size_hl[1])
W3=np.random.randn(size_ol, size_hl[1]) * np.sqrt(1/size_ol)

In [22]:
#the expression
def sigmoid(x):
    return 1/(1 + np.exp(-x))
def dsigmoid(x):
    return (np.exp(-x))/((np.exp(-x)+1)**2)

In [23]:
def feet_forward_propogation():
    A0=x_train.T
    Z1=np.dot(W1,A0)
    A1=sigmoid(Z1)
    Z2=np.dot(W2,A1)
    A2=sigmoid(Z2)
    Z3=np.dot(W3,A2)
    A3=sigmoid(Z3)
    return A0,A1,A2,A3,Z1,Z2,Z3

In [24]:
def backward_propogation(A3,A2,A1,A0,Z1,Z2,a):
    global W3,W2,W1
    dZ3=A3-y_train.T
    dW3 = np.dot(dZ3, A2.T)
    err = np.dot(W3.T, dZ3) * dsigmoid(Z2)
    dW2 = np.dot(err, A1.T)
    err = np.dot(W2.T, err) * dsigmoid(Z1)
    dW1 = np.dot(err, A0.T)
    dZ2 = np.multiply(np.dot(W3.T,dZ3), 1-np.power(A2,2))
    dZ1 = np.multiply(np.dot(W2.T,dZ2), 1-np.power(A1,2))
    W3 -= a*dW3
    W2 -= a*dW2
    W1 -= a*dW1

In [25]:
#cost function 
def cross_entropy_cost(A3):
    num = y_train.shape[1]
    logprobs=np.multiply(np.log(A3),y_train.T) + np.multiply((1-y_train.T),np.log(1-A3))
    cost = - np.sum(logprobs) / num
    cost = float(np.squeeze(cost))
    return cost


In [26]:
#training the neural network
for i in range(1000):
    a=0.00003
    A0,A1,A2,A3,Z1,Z2,Z3=feet_forward_propogation()
    cost = cross_entropy_cost(A3)
    backward_propogation(A3,A2,A1,A0,Z1,Z2, a)
    print("Cost after iteration ",i," is ", cost)

Cost after iteration  0  is  60107.10587878492
Cost after iteration  1  is  79894.03311471659
Cost after iteration  2  is  52827.85886118272
Cost after iteration  3  is  49833.921727045206
Cost after iteration  4  is  34562.67616831167
Cost after iteration  5  is  29325.95874412363
Cost after iteration  6  is  26182.149073966393
Cost after iteration  7  is  22263.493674750716
Cost after iteration  8  is  22310.814270293795
Cost after iteration  9  is  21148.921525700225
Cost after iteration  10  is  21718.511670553464
Cost after iteration  11  is  21793.20727702276
Cost after iteration  12  is  21913.960858929033
Cost after iteration  13  is  21076.48448917514
Cost after iteration  14  is  23669.875380924044
Cost after iteration  15  is  20317.550809673194
Cost after iteration  16  is  20874.872205986892
Cost after iteration  17  is  21834.824348972954
Cost after iteration  18  is  21377.838298455208
Cost after iteration  19  is  20848.081456094766
Cost after iteration  20  is  20939.9

In [27]:
#accuracy
A0=x_train.T
Z1=np.dot(W1,A0)
A1=sigmoid(Z1)
Z2=np.dot(W2,A1)
A2=sigmoid(Z2)
Z3=np.dot(W3,A2)
A3=sigmoid(Z3)
A3=A3.T
for x in A3:
    ma=np.argmax(x)
    for y in range(len(x)):
        if y==ma:
            x[y]=1
        else:
            x[y]=0
A3=A3.T
acc1=0 #accuracy of the nn for training data
acc2=0 #accuracy of the nn for testing data
for i in range(len(A3)):
    for j in range(len(A3[0])):
        if A3[i][j]==y_train[j][i]:
            acc1+=1
acc1/=len(A3)*len(A3[0])
A0=x_test.T
Z1=np.dot(W1,A0)
A1=sigmoid(Z1)
Z2=np.dot(W2,A1)
A2=sigmoid(Z2)
Z3=np.dot(W3,A2)
A3=sigmoid(Z3)
A3=A3.T
for x in A3:
    ma=np.argmax(x)
    for y in range(len(x)):
        if y==ma:
            x[y]=1
        else:
            x[y]=0
A3=A3.T
for i in range(len(A3)):
    for j in range(len(A3[0])):
        if A3[i][j]==y_test[j][i]:
            acc2+=1
acc2/=len(A3)*len(A3[0])

In [29]:
print(acc1)
print(acc2)

0.9940033613445378
0.9928


A Neural Network using Feed forward and Back propogation with high accuracy